# Retrival Based two Agent: (OPENAI) 1. REASONING AGENT + 2. RETRIEVER AGENT(RAG)

🟢 OpenAI — model = "gpt-4o-mini"

Retriever Agent handles document search via vectors.

Reasoning Agent uses GPT-4o-mini (cloud).

Strong reasoning and instruction following.

Easy integration with AutoGen / LangGraph.

Best for enterprise-grade RAG systems.

In [9]:
# =========================================================
# TWO-AGENT RAG — OPENAI (gpt-4o-mini)
# =========================================================

from openai import OpenAI
from pypdf import PdfReader
from sentence_transformers import SentenceTransformer
import chromadb

# =========================================================
# AGENT 2: OPENAI REASONING AGENT
# =========================================================
OPENAI_API_KEY = "--your openai Key--"

client = OpenAI(api_key=OPENAI_API_KEY)

class OpenAIAgent:
    def answer(self, context, question):
        prompt = f"""
You are a document-grounded AI assistant.

Rules:
- Answer ONLY using the provided context.
- If the answer is not in the context, say exactly:
  Not found in the document.

Context:
{context}

Question:
{question}
"""
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "user", "content": prompt}
            ],
            temperature=0
        )
        return response.choices[0].message.content


# =========================================================
# AGENT 1: RETRIEVER AGENT
# =========================================================
class RetrieverAgent:
    def __init__(self, pdf_path):
        self.embedder = SentenceTransformer("all-MiniLM-L6-v2")
        self.client = chromadb.Client()
        self.collection = self.client.get_or_create_collection(
            name="openai-docs"
        )
        self._ingest_pdf(pdf_path)

    def _ingest_pdf(self, pdf_path):
        reader = PdfReader(pdf_path)
        text = "\n".join(page.extract_text() for page in reader.pages)

        words = text.split()
        chunks = [
            " ".join(words[i:i + 300])
            for i in range(0, len(words), 300)
        ]

        for i, chunk in enumerate(chunks):
            self.collection.add(
                documents=[chunk],
                ids=[str(i)],
                embeddings=[self.embedder.encode(chunk).tolist()]
            )

    def retrieve(self, question, k=3):
        q_embedding = self.embedder.encode(question).tolist()
        results = self.collection.query(
            query_embeddings=[q_embedding],
            n_results=k
        )
        return "\n".join(results["documents"][0])


# =========================================================
# ORCHESTRATOR
# =========================================================
retriever_agent = RetrieverAgent("Ai_Test_Book.pdf")
openai_agent = OpenAIAgent()

# =========================================================
# LIMIT TO ONLY 3 QUESTIONS
# =========================================================
MAX_QUESTIONS = 3
question_count = 0

questions = [
    "What is Artificial Intelligence?",
    "What is your salary?",
    "Explain agent-based AI systems."
]

# =========================================================
# RUN TWO-AGENT RAG
# =========================================================
for question in questions:
    if question_count >= MAX_QUESTIONS:
        print("❌ Question limit reached. Session ended.")
        break

    print("\n--------------------------------")
    print(f"Question: {question}")

    # AGENT 1 → RETRIEVE
    context = retriever_agent.retrieve(question)

    # AGENT 2 → REASON
    answer = openai_agent.answer(context, question)

    print(f"Answer: {answer}")

    question_count += 1



--------------------------------
Question: What is Artificial Intelligence?
Answer: Artificial Intelligence (AI) is a branch of computer science that focuses on building systems capable of performing tasks that normally require human intelligence. These tasks include reasoning, learning, problem-solving, perception, and language understanding. The term Artificial Intelligence was first introduced by John McCarthy in 1956 during the Dartmouth Conference. Since then, AI has evolved from simple rule-based systems to advanced machine learning and deep learning models. AI systems can be broadly classified into Narrow AI, which is designed for a specific task, and General AI, which is capable of performing any intellectual task a human can do (still theoretical). AI is widely used in healthcare, finance, transportation, education, and research.

--------------------------------
Question: What is your salary?
Answer: Not found in the document.

--------------------------------
Question: Expl